<a href="https://colab.research.google.com/github/alfbrynn/MachineLearning/blob/main/UTS_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# 0. Instalasi dependensi (jalankan jika perlu)
# ================================
!pip install annoy scikit-learn pandas matplotlib seaborn
# Jika ingin load langsung dari Kaggle via kagglehub (opsional):
# !pip install kagglehub[pandas-datasets]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551808 sha256=7f77dc6744b4d3a0c3cecbbbcb47aad3e372977404a82b689e82c3c5e10e4f03
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy
